# **MODELO DE CLASIFICACIÓN RANDOM FOREST OPTIMIZADO PARA RAM** #

Este script implementa un modelo de clasificación con Random Forest usando un enfoque optimizado para grandes volúmenes de datos. Se prioriza la eficiencia en el uso de memoria RAM y la simplicidad de codificación. El objetivo es predecir la variable categórica 'RENDIMIENTO_GLOBAL'

### **FASE 1:Carga de librerías y función de procesamiento avanzado**

In [1]:
#Carga de librerías

import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


In [2]:
#Función de procesamiento avanzado

def preprocesamiento_avanzado_definitivo(df_input):
    df = df_input.copy()

    #Eliminar columnas con un solo valor
    for col in df.columns:
        if df[col].nunique(dropna=False) <= 1:
            df.drop(columns=[col], inplace=True)

    #Rellenar valores nulos en columnas categóricas con la moda.
    for col in df.select_dtypes(include=['object']).columns:
        if col != 'RENDIMIENTO_GLOBAL':
            df[col].fillna(df[col].mode()[0], inplace=True)

    #Identificar automáticamente columnas binarias y convertirlas a valores numéricos (1 y 0).
    si_no_map = {'Si': 1, 'No': 0, 'S': 1, 'N': 0}
    for col in df.select_dtypes(include=['object']).columns:
        unique_vals = set(df[col].dropna().unique())
        if unique_vals.issubset(si_no_map.keys()):
            print(f"  -> Columna binaria encontrada: '{col}'. Convirtiendo a 1/0.")
            df[col] = df[col].map(si_no_map).astype('Int8')

    #Mapeos manuales restantes para columnas ordinales
    edu_map = {'Ninguno': 0, 'No sabe': 0, 'Primaria incompleta': 1, 'Primaria completa': 2,
               'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
               'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6, 'Postgrado': 7}
    if 'FAMI_EDUCACIONPADRE' in df.columns:
        df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].map(edu_map)
    if 'FAMI_EDUCACIONMADRE' in df.columns:
        df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].map(edu_map)
    if 'ESTU_VALORMATRICULAUNIVERSIDAD' in df.columns:
        df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map({
            'No pagó matrícula': 0, 'Menos de 500 mil': 1, 'Entre 500 mil y menos de 1 millón': 2,
            'Entre 1 millón y menos de 2.5 millones': 3, 'Entre 2.5 millones y menos de 4 millones': 4,
            'Entre 4 millones y menos de 5.5 millones': 5, 'Entre 5.5 millones y menos de 7 millones': 6,
            'Más de 7 millones': 7})
    if 'ESTU_HORASSEMANATRABAJA' in df.columns:
        df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].map({
            '0': 0, 'Menos de 10 horas': 1, 'Entre 11 y 20 horas': 2, 'Entre 21 y 30 horas': 3, 'Más de 30 horas': 4})
    if 'FAMI_ESTRATOVIVIENDA' in df.columns:
        df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].str.replace('Estrato ', '').str.replace('Sin Estrato', '0').astype(np.int8)

    #Feature Engineering para columnas ya convertidas a número
    df['INDICE_SOCIOECONOMICO'] = (df['FAMI_TIENEINTERNET'] + df['FAMI_TIENELAVADORA'] + df['FAMI_TIENECOMPUTADOR'] + df['FAMI_TIENEAUTOMOVIL']).astype(np.int8)
    df['PROMEDIO_EDU_PADRES'] = (df['FAMI_EDUCACIONPADRE'] + df['FAMI_EDUCACIONMADRE']).astype(np.float32) / 2.0
    df.drop(['FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE'], axis=1, inplace=True, errors='ignore')

    #Uso de One-Hot Encoding para columnas categóricas con muchas clases
    df = pd.get_dummies(df, columns=['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO'], dummy_na=False, dtype=np.int8)

    return df


### **FASE 2: Carga y prepocesamientos de los datos**

In [3]:
#Carga y prepocesamientos de los datos

#Se leen los respectivos archivos csv
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

#Se guardan la variable a predecir (RENDIMIENTO GLOBAL) y el ID
target_col = df_train['RENDIMIENTO_GLOBAL']
test_ids = df_test['ID']

#Se elimina la columna a predecir en el dataset de train para dejar solo variables independientes
df_train = df_train.drop(columns=['RENDIMIENTO_GLOBAL'])

#Se aplica la función de limpieza y codificación
train_processed = preprocesamiento_avanzado_definitivo(df_train)
test_processed = preprocesamiento_avanzado_definitivo(df_test)

#Se asegura que ambos conjuntos tenga las mismas columnas y el mismo orden
train_final, test_final = train_processed.align(test_processed, join='inner', axis=1, fill_value=0)

#Se codifica la variable objetivo con números enteros para entrenamiento supervisado
le = LabelEncoder()
y = le.fit_transform(target_col)

#Se define X como el conjunto de características para entrenamiento y se elimina la columna ID ya que no es útil para el modelo
X = train_final.drop(columns=['ID'], errors='ignore')
X_submission = test_final.drop(columns=['ID'], errors='ignore')


/tmp/ipython-input-2-3615227931.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


  -> Columna binaria encontrada: 'FAMI_TIENEINTERNET'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENELAVADORA'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENEAUTOMOVIL'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'ESTU_PRIVADO_LIBERTAD'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'ESTU_PAGOMATRICULAPROPIO'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENECOMPUTADOR'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENEINTERNET.1'. Convirtiendo a 1/0.


/tmp/ipython-input-2-3615227931.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


  -> Columna binaria encontrada: 'FAMI_TIENEINTERNET'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENELAVADORA'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENEAUTOMOVIL'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'ESTU_PRIVADO_LIBERTAD'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'ESTU_PAGOMATRICULAPROPIO'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENECOMPUTADOR'. Convirtiendo a 1/0.
  -> Columna binaria encontrada: 'FAMI_TIENEINTERNET.1'. Convirtiendo a 1/0.


### **FASE 3: Entrenamiento del modelo Random Forest**

In [4]:
#División de conjunto de datos
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Set de entrenamiento: {X_train.shape}, Set de validación: {X_val.shape}")

#Entrenamiento del modelo con sus respectivos hiperparámetros
model_rf = RandomForestClassifier(n_estimators=200, max_depth=12, random_state=42, n_jobs=-1, min_samples_leaf=5)
model_rf.fit(X_train, y_train)

#Predicciones en el conjunto de validación
preds_val = model_rf.predict(X_val)
accuracy = accuracy_score(y_val, preds_val)

#S calcula la exactitud (accuracy)
print(f"Accuracy en validación: {accuracy:.4f}")


Set de entrenamiento: (554000, 957), Set de validación: (138500, 957)
Accuracy en validación: 0.3890


### **FASE 4: Predicción final y generación del archivo Submission**

In [5]:
#Se usa el modelo entrenado para hacer predicciones sobre los datos del archivo test.csv

preds_submission = model_rf.predict(X_submission)
submission_labels = le.inverse_transform(preds_submission)

#Se crea un DataFrame con los ID origniales y las predicciones
submission_df = pd.DataFrame({"ID": test_ids,"RENDIMIENTO_GLOBAL": submission_labels})

#Se guarda el archivo en formato csv
submission_df.to_csv("submission.csv", index=False)

print("Archivo 'submission.csv' generado")


Archivo 'submission.csv' generado
